In [41]:
with open('/Users/admin-tascott/Documents/GitHub/openai_eis_key.txt', encoding="utf-8") as f:
    key = f.read()
import os
os.environ["OPENAI_API_KEY"] = key
#!pip install pypdf
#!pip install faiss-cpu
#!pip install langchain
#### setup for query, feed in individual files
#### note that future improvement would be to load docs as a library and query across library
### presumably this would be much faster, for for now I was getting token warnings (to many)
#### but single-file-at-a-time approach does work

# if overwrite = T, then redo prior extractions
overwrite = False
import re
import langchain.document_loaders
from langchain.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import pandas as pd
from io import StringIO
import nltk
### find the meeting files in these subdirectories
import os, fnmatch
file_list = []
original_files = '../../data/raw/_firstjpeg/'
files = fnmatch.filter(os.listdir(original_files), '*.jpeg')
for f in files:
    file_list.append(f)
file_list.sort()
import random
#random.shuffle(file_list)
savedir = '../../data/ocr_collected'
if os.path.exists(savedir)==False:
    os.mkdir(savedir)
#!mkdir ../../data/intermediate_products
tagged_pages = pd.read_csv('../../data/intermediate_products/zeroshot_form_contract.csv')

contract_files = tagged_pages[tagged_pages['prose in single column layout']>0.5]

from openai import OpenAI
import base64
client = OpenAI()

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')
# Path to your image
image_loc = "../../data/raw/_firstjpeg/"
df_files = [f for f in contract_files['file']]
loc = '../../data/intermediate_products/json_contracts/'
!mkdir loc
df_need = set([re.sub('jpeg','json',f) for f in df_files]).difference(set(os.listdir(loc)))
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {key}"
}
import requests
import json
prompt = """This image is the first page of a contract with a stage agency. 
Record the type of agreement and whether it the agreement is new, an amendment, an extension, or a renewal.
Itemize contract data such as the involved parties, amount, relevant dates, and whether the scope of work is reduced, expanded, or remains the same.
Provide the results as a json file"""


mkdir: loc: File exists


In [42]:
test = [d for d in df_need][:10]

In [43]:
for d in test:
#for d in df_need:
    # Getting the base64 string
    jpeg_file = re.sub('json','jpeg',d)
    image = '{}/{}'.format(image_loc,jpeg_file)
    base64_image = encode_image(image)
    payload = {
      "model": "gpt-4o",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": prompt
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}",
                "detail": "high"
              }
            }
          ]
        }
      ],
      "max_tokens": 1000
    }
    
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    jsonfile = d
    with open('{}/{}'.format(loc,jsonfile), 'w', encoding='utf-8') as f:
        json.dump(response.json(), f, ensure_ascii=False, indent=4)